<a style="float:left;" href="https://colab.research.google.com/github/ClaudeCoulombe/VIARENA/blob/master/Labos/Lab-MNIST/Identification_ChiffresManuscrits_MNIST-ResConv.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<br/>
### Rappel - Fonctionnement d'un carnet web iPython

* Pour exécuter le code contenu dans une cellule d'un carnet iPython, cliquez dans la cellule et faites (⇧↵, shift-enter);
* Le code d'un carnet iPython s'exécute séquentiellement de haut en bas de la page. Souvent, l'importation d'une bibliothèque Python ou l'initialisation d'une variable est préalable à l'exécution d'une cellule située plus bas. Il est donc recommandé d'exécuter les cellules en séquence. Enfin, méfiez-vous des retours en arrière qui peuvent réinitialiser certaines variables;
* Pour obtenir de l'information sur une fonction, utilisez la commande Python `help(`"nom de la fonction"`)`

# Identification de chiffres manuscrits - jeu de données MNIST
## Réseau convolutif - LeNet

#### Inspiration: 
 
Richmond Alake - <a href="https://towardsdatascience.com/understanding-and-implementing-lenet-5-cnn-architecture-deep-learning-a2d531ebc342" target='_blank'>Understanding and Implementing LeNet-5 CNN Architecture</a> - 25 juin 2020

Yann LeCun, Léon Bottou, Yoshua Bengio et Patrick Haffner <a href="http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf" target='_blank'>Gradient-Based Learning Applied to Document Recognition</a> - 1998



## Importation des bibliothèques Python

In [ ]:
import os
import matplotlib.pyplot as plt

import tensorflow as tf
print("TensorFlow version:",tf.__version__)
from tensorflow import keras
print("Keras version:",keras.__version__)

## Jeu de données - chiffres manuscrits MNIST

Le jeu de données MNIST (Modified National Institute of Standards and Technology) comporte 60,000 images en tons de gris de 28×28 pixels de chiffres manuscrits étiquetés de 0 à 9. Site web: http://yann.lecun.com/exdb/mnist/

Il est incorporé dans keras.datasets 

### Lecture des données
### Séparation entre jeu de données d'entraînement et jeux de données de test

In [ ]:
# le jeu de données MNIST
from keras.datasets import mnist

dic_noms_classe = { 
    0 : "0",
    1 : "1",
    2 : "2",
    3 : "3",
    4 : "4",
    5 : "5",
    6 : "6",
    7 : "7",
    8 : "8",
    9 : "9",
}

# lire le jeu de données MNIST et le diviser entre
# les données d'entrainement et les données de test
# MNIST est déjà divisé en un jeu de données d'entraînement (les 60 000 premières images) 
# et un jeu de données de test (les 10 000 dernières images).
(attributs_entrainement, classes_cibles_entrainement), (attributs_test, classes_cibles_test) = mnist.load_data()


## Exploration des données

### Portrait des données

In [ ]:
# Portrait des données 
print()
print('Entraînement: attributs=%s, classes=%s' % (attributs_entrainement.shape, classes_cibles_entrainement.shape))
print('Test: attributs=%s, classes=%s' % (attributs_test.shape, classes_cibles_test.shape))

In [ ]:
attributs_entrainement[0].shape

### Visualisation de données

In [ ]:
# Afficher les 24 premières images
print()
print("Quelques images avec leur étiquette de classe-cible...")
%matplotlib inline
# définir subplot
fig, axes = plt.subplots(nrows=4,ncols=6,figsize=(13,10))
for i_rangee in range(0,4):
    for i_colonne in range(0,6):
        axes[i_rangee,i_colonne].set_title(dic_noms_classe[int(classes_cibles_entrainement[i_rangee*6+i_colonne])],
                                           fontsize=10)
        axes[i_rangee,i_colonne].imshow(attributs_entrainement[i_rangee*6+i_colonne],cmap='gray')
plt.show()

## Préparation des données

### Conversion des étiquettes-cibles en vecteurs binaires à un bit discriminant

In [ ]:
# Conversion des étiquettes-cibles en vecteurs binaires à un bit discriminant
from tensorflow.keras.utils import to_categorical
classes_cibles_entrainement = to_categorical(classes_cibles_entrainement)
classes_cibles_test = to_categorical(classes_cibles_test)
print("Conversion des étiquettes-cibles en vecteurs binaires terminée!")

### Normalisation des images

In [ ]:
# Normalisation
def normalisation(entrainement, test):
    # convertir de nombres entiers à nombres décimaux
    entrainement_normalise = entrainement.astype('float32')
    test_normalise = test.astype('float32')
    # normalisation à un nombre entre 0 et 1
    entrainement_normalise = entrainement_normalise / 255.0
    test_normalise = test_normalise / 255.0
    return entrainement_normalise, test_normalise

attributs_entrainement, attributs_test = normalisation(attributs_entrainement, attributs_test)

print("Normalisation des images terminée!")

## Construction d'un réseau convolutif - LeNet

<img src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@LeNet-52.png">

In [ ]:
# Construction du modèle

from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense

print("Création d'un modèle LeNet...")

# dimension_entree = (nombre_attributs,)
nombre_classes_cibles = 10

increment_convolution = (1,1)
increment_echantillonnage = (2,2)

leNet = Sequential()

# Apprentissage et extraction des attributs
# C1
leNet.add(Conv2D(filters=6, 
                 kernel_size=(5,5), 
                 strides = increment_convolution,
                 padding = 'same',
                 activation='tanh', 
                 input_shape=(28,28,1)))
# S2
leNet.add(AveragePooling2D(pool_size=(2, 2),
                           strides=increment_echantillonnage))

# C3
leNet.add(Conv2D(filters=16, 
                 kernel_size=(5, 5),
                 strides = increment_convolution,
                 padding = 'valid',
                 activation='tanh'))
# S4
leNet.add(AveragePooling2D(pool_size=(2, 2),
                           strides=increment_echantillonnage))

# Classification des images
leNet.add(Flatten())

# C5
leNet.add(Dense(units=120, 
                activation='tanh'))
# F6
leNet.add(Dense(units=84, 
                activation='relu'))

# Couche de sortie
leNet.add(Dense(units=nombre_classes_cibles,
                activation = 'softmax'))

print()
print("Description du modèle de base:")
leNet.summary()

### Compilation du modèle

In [ ]:
print()
print("Compilation du modèle...")

leNet.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

print("Modèle compilé!")

## Entraînement du modèle

In [ ]:
# Entraînement du modèle

print()
print("Entraînement du modèle...")

batch_size = 128
epochs = 20
taille_jeu_validation = 0.1

# Les images MNIST sont de 28x28 pixels, ce qui est plus petit que ce que LeNet-5 
# attend de 32x32 pixels. Une solution simple consiste simplement à remplir les images 
# avec des zéros pour amener la taille des images MNIST jusqu'à 32x32 pixels.

attributs_entrainement = attributs_entrainement.reshape(attributs_entrainement.shape[0], 28, 28, 1)
attributs_test = attributs_test.reshape(attributs_test.shape[0], 28, 28, 1)

traces_entrainement = leNet.fit(attributs_entrainement,
                                classes_cibles_entrainement,
                                batch_size=batch_size,
                                epochs=epochs,
                                validation_split=taille_jeu_validation)

## Évaluation du modèle avec un jeu de données test

In [ ]:
# Évaluation du modèle

print()
print("Évaluation du modèle...")

resultats = leNet.evaluate(attributs_test, classes_cibles_test, verbose=0)
print("Exactitude test: {:.2f}%".format(resultats[1]*100))

**Note:** On constate un gain d'environ 5% avec l'utilisation d'un réseau convolutif comme l'antique LeNet et l'exactitude obtenue avec un perceptron multicouche. Cette différence démontre l'avantage des réseaux convolutifs pour la reconnaissance d'images même simples comme des chiffres manuscrits en noir et blanc. 

In [ ]:
print("Exécution carnet IPython terminée!")